## ANALIZA PODATKOV - KNJIGE


Najprej je potrebno naložiti podatke z interneta. Ker smo podatke že naložili in obdelali lokalno, tega ne potrebujemo posebej zaganjati.

In [ ]:
import prevedi_in_shrani_podatke

# prevede in shrani podatke z interneta
prevedi_in_shrani_podatke.save_web_data()

In [ ]:
import obdelaj_podatke

# obdela shranjene podatke in jih zapise v csv datoteko
obdelaj_podatke.pripravi_podatke_za_csv()

# TABELA S PODATKI

Za prikaz vhodnih podatkov uporabimo knjižnico pandas.

In [10]:
# naložimo paket
import pandas as pd

# ker bomo delali z velikimi razpredelnicami, povemo, da naj se vedno izpiše le 10 vrstic
pd.set_option('display.max_rows', 10)

# izberemo interaktivni "notebook" stil risanja
%matplotlib inline

# naložimo razpredelnico, s katero bomo delali
KNJIGE = pd.read_csv('koncni_podatki/podatki_o_knjigi.csv', index_col='naslov', encoding='utf-8')

In [11]:
KNJIGE

,avtor,dolzina,zvrst,o_pisatelju
naslov,,,,
"Ne povej, kaj si sanjala - zvočna knjiga na USB ključu",Feri Lainšček,131.0,roman,"Pisatelj, pesnik, dramatik in scenarist Feri L..."
Terenske raziskave ukrajinskega seksa,Oksana Zabužko,168.0,roman,Oksana Zabužko (1960) je ugledna ukrajinska fi...
Mitrejin koder,Miha Remec,204.0,roman,"Miha Remec (1922, Ptuj) je pisec znanstvene fa..."
Moje leto v Nikogaršnjem zalivu,Peter Handke,NaN,roman,Peter Handke (1942) je eden največjih živečih ...
Dežela ZOO,Dušan Čater,200.0,roman,"Du&scaron;an Čater (1968) je pisatelj, prevaja..."
...,...,...,...,...
Veronikin prt,Josip Osti,116.0,poezija,"Josip Osti (1945-;2021) je bil pesnik, pisatel..."
"Oče, tvoje oči gorijo",Ivo Svetina,116.0,poezija,"Ivo Svetina (1948, Ljubljana) je eden pomembne..."
Pod kamni plešem,Jure Vuga,77.0,poezija,"Jure Vuga (1983, Koper) je profesor umetnostne..."
